In [0]:
dbutils.widgets.text("catalog", "default", "Catalog")
dbutils.widgets.text("schema", "public", "Schema")

In [0]:
%pip install dbldatagen faker

In [0]:
from dbldatagen import DataGenerator, fakerText
from faker.providers import internet

shuffle_partitions_requested = 8
partitions_requested = 8
data_rows = 100000

# partition parameters etc.
spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

my_word_list = [ "HIGH", "MEDIUM", "LOW" ]

fakerDataspec = (DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
            .withColumn("name", text=fakerText("name") )
            .withColumn("email", text=fakerText("ascii_company_email") )
            .withColumn("clearance", text=fakerText("word", ext_word_list=my_word_list))
            )
dfFakerOnly = fakerDataspec.build()
display(dfFakerOnly)

In [0]:
import random
import string
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def unique_sorted_upper_str():
    n = random.randint(0, 4)
    chars = random.sample(string.ascii_uppercase[0:5], n)
    return ''.join(sorted(chars))

unique_str_udf = udf(unique_sorted_upper_str, StringType())


In [0]:
unique_sorted_upper_str()

In [0]:
df = dfFakerOnly.withColumn("compartment", unique_str_udf())
display(df)

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
users_path = "{}.{}.users".format(catalog, schema)
print(users_path)

In [0]:

df.write.format("delta").mode("overwrite").saveAsTable(users_path)